# Analýza hlavních komponent (PCA)

V tomto cvičení budeme vycházet ze stejného datového setu, jako při výpočtu lineární regrese. 

Připomeňme si, že na základě korelační matice jsme do modelu vybrali proměnné RM, LSTAT a vysvětlovanou proměnnou MEDV.

Vytvořený model měl na trénovacích a testovacích datech R2 skóre přibližně 0,65.

Pomocí PCA se pokusíme dosáhnout lepšího výsledku, tedy vytvořit model, který bude lépe předpovídat.

## Načtení a analýza dat

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import numpy as np

In [ ]:
data = pd.read_csv ("..\\dataset\\HousingData.csv")

In [ ]:
data=data.dropna()

In [ ]:
data.head()

In [ ]:
data.describe()

Měřítka proměnných se od sebe velmi liší, a proto budeme muset data standardizovat.

## Lineární model všech proměnných bez úprav
Vytvoříme kontrolní lineární model se všemi proměnnými. 

Cílem je mít vzorový model, který budeme porovnávat s vylepšeními.

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

Data rozdělíme na trénovací a testovací

In [ ]:
X = np.array(data.drop('MEDV',axis=1))
y = np.array(data['MEDV'])
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

Vytvoření lineárního modelu

In [ ]:
lr = LinearRegression()
lr.fit(X_train, y_train)

Vyhodnocení modelu na trénovacích datech

In [ ]:
y_pred = lr.predict(X_train)
r2 = r2_score(y_train, y_pred)
rmse = np.sqrt(mean_squared_error(y_train, y_pred))

print(f"R2 score: {r2}")
print(f"RMSE: {rmse}")

Vyhodnocení modelu na testovacích datech

In [ ]:
y_pred = lr.predict(X_test)
r2 = r2_score(y_test, y_pred)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))

print(f"R2 score: {r2}")
print(f"RMSE: {rmse}")

## Korelace
Opět provedeme korelační analýzu a hledáme lineární závislosti mezi proměnnými.

In [ ]:
fig = plt.figure(figsize=(8,8))
ax = fig.add_subplot(111)
sns.heatmap(data.corr(),annot=True)

**Korelační matice** obsahuje Pearsonovy korelační koeficienty mezi všemi páry proměnných. Hodnoty se pohybují od -1 do 1.
* 1 → silná pozitivní lineární korelace
* -1 → silná negativní lineární korelace
* 0 → žádná lineární korelace

**Multikolinearita** nastává, když jsou predikční proměnné silně korelovány mezi sebou (např. DIS s INUDS, INOX, AGE).

Problém:
* Odhady regresních koeficientů jsou nestabilní a citlivé na malé změny dat.
* Interpretace jednotlivých koeficientů ztrácí smysl, protože nelze izolovat efekt jedné proměnné při fixních ostatních, pokud jsou navzájem korelované.
* Zvyšuje standardní chybu koeficientů, což snižuje statistickou významnost.


Důležitý je poslední řádek, který nám ukazuje lineární korelaci vysvětlujících proměnných a vysvětlované proměnné MEDV. Zdá se, že naše cílová proměnná je vysoce korelovaná s LSTAT a RM, což dává smysl, protože tyto dva faktory jsou velmi důležité pro tvorbu cen domů.  Je zde také mnoho multikolinearity.

Obvyklá interpretace regresního koeficientu je taková, že poskytuje odhad účinku jednotkové změny nezávislé proměnné na závislou proměnnou při zachování ostatních proměnných jako konstantních. V případě multikolinearity, to ale nemůžeme říci. Pokud je X1 v daném souboru dat silně korelována s jinou nezávislou proměnnou, X2, pak máme soubor pozorování, pro který X1 a X2 mají určitý lineární stochastický vztah. Nemůžeme tak zajistit při změně proměnné X1 X2 zůstane konstantní.

## Faktor inflace rozptylu (VIF) 

**Faktor inflace rozptylu (VIF)** detekuje multikolinearitu v regresní analýze. 

Její přítomnost může negativně ovlivnit výsledky regrese. VIF odhaduje, jak moc je rozptyl regresního koeficientu nadsazen v důsledku multikolinearity v modelu.

VIF=1/(1−R^2)

Kde R^2 je koeficient determinace. 

Zjednodušeně řečeno, je to podíl rozptylu nezávislé proměnné, který je vysvětlen závislou proměnnou. 

Provedeme tedy lineární regresi s použitím každé proměnné jako cíle a ostatních jako prediktorů a vypočítáme R^2 a poté pro ně vypočítáme VIF.

Pokud je VIF < 4, lze proměnnou použít, v opačném případě musíme najít způsob, jak odstranit kolinearitu těchto proměnných.

In [ ]:
vifdf = []
for i in data.columns:
    X = np.array(data.drop(i,axis=1))
    y = np.array(data[i])
    lr = LinearRegression()
    lr.fit(X,y)
    y_pred = lr.predict(X)
    r2 = r2_score(y,y_pred)
    vif = 1/(1-r2)
    vifdf.append((i,vif))

vifdf = pd.DataFrame(vifdf,columns=['Features','Variance Inflation Factor'])
vifdf.sort_values(by='Variance Inflation Factor')

Vidíme, že téměř polovina proměnných má  hodnotu VIF vyšší, nebo blízkou 4. TAX a RAD mají VIF téměř dvakrát vyšší, než je naše prahová hodnota.

Bude tedy vhodné vyřešit multikolinearitu. To lze dělat více způsoby:
* Odstranění korelovaných proměnných → vybrat jen jednu z páru silně korelovaných.
* Principal Component Analysis (PCA) → transformovat prediktory do ne-korelovaných komponent.
* Regularizace (Ridge, Lasso) → potlačuje vliv kolinearity a stabilizuje model.

My se podíváme na PCA.

## Standardizace dat
Prvním krokem je standardizace dat, tak aby všechny proměnné měly střední hodnotu okolo 0. Pak jejich vliv na výstupní proměnnou budou podobný.

Podíváme se na data před standardizací.

In [ ]:
pos = 1
fig = plt.figure(figsize=(8, 12))
for i in data.columns:
    ax = fig.add_subplot(7,2, pos)
    pos = pos + 1
    sns.histplot(data[i],ax=ax, kde=True)

Provedeme z-standardization pomocí funkce rescale.

In [ ]:
def rescale(X):
    mean = X.mean()
    std = X.std()
    scaled_X = [(i - mean)/std for i in X]
    return pd.Series(scaled_X)

Vytvoříme si nový standardizovaný dataset data_std.

In [ ]:
data_std = pd.DataFrame(columns=data.columns)
for i in data.columns:
    data_std[i] = rescale(data[i])

Pro kontrolu si vypíšeme základní statistiku.

In [ ]:
data_std.describe()

Zobrazení distribuce hodnot s odhadem distribuční funkce.

Tvar distribuce nových proměnných je stejný jako u původních proměnných. Pouze jejich střední hodnota je nyní 0.

In [ ]:
pos = 1
fig = plt.figure(figsize=(8,12))
for i in data_std.columns:
    ax = fig.add_subplot(7,2, pos)
    pos = pos + 1
    sns.histplot(data_std[i],ax=ax, kde=True)

Podíváme se na korelaci standardizovaných dat. Ta zůstává stejná.

In [ ]:
fig = plt.figure(figsize=(8,8))
ax = fig.add_subplot(111)
sns.heatmap(data_std.corr(),annot=True)

## PCA

### Myšlenka
* Pokud máme hodně vzájemně korelovaných proměnných, tak je v datech skrytá redundance.
* PCA tuto redundanci odstraní tím, že původní proměnné převede na nové, nekorelované proměnné = hlavní komponenty.
* Tyto komponenty jsou lineární kombinací původních proměnných.


### Jak PCA funguje? (intuice)
* Najde směr s největším rozptylem dat (1. hlavní komponenta).
* Najde druhý směr s co největším rozptylem, ale ortogonální k prvnímu (2. hlavní komponenta).
* Pokračuje, dokud nevyčerpá všechny dimenze.
* Výsledek:
    * Hlavní komponenty jsou nekorelované.
    * První komponenty vysvětlují většinu variability v datech.


### K čemu PCA slouží
* Odstranění multikolinearity → komponenty jsou ortogonální → žádná kolinearita.
* Redukce dimenzionality → necháme si jen prvních pár komponent, které vysvětlí např. 90–95 % variability.
* Vizualizace → složitá data z mnoha proměnných lze vykreslit do 2D/3D prostoru.


PCA je citlivá na měřítko proměnných. Proto se před aplikací PCA obvykle dělá standardizace.

PCA nebudeme psát ručně, ale použijeme její implementaci z knihovny.

In [ ]:
from sklearn.decomposition import PCA

Počet PCA komponent bude 13 stejně jako vstupních parametrů

Výstupní MEDV musíme ze vstupu do PCA odstranit.

In [ ]:
pca = PCA(n_components=13)
X = data_std.drop('MEDV',axis=1)
X_pca = pca.fit_transform(X)

Nyní si vytvoříme nový data s hlavními komponentami jako vstupní proměnné a MEDV jako výstupní proměnnou.

In [ ]:
data_std_pca = pd.DataFrame(X_pca,columns=['PCA1','PCA2','PCA3','PCA4','PCA5','PCA6','PCA7','PCA8','PCA9','PCA10','PCA11','PCA12','PCA13'])
data_std_pca['MEDV'] = data_std['MEDV']

PCA měla redukovat multikolinearitu. Tak si to zkontrolujeme.

In [ ]:
fig = plt.figure(figsize=(12,8))
ax = fig.add_subplot(111)
sns.heatmap(data_std_pca.corr(),annot=True)

V korelační matici je vidět, že PCA komponenty nejsou na sobě závislé.

MEDV je lineárně závislý na prvních 3 PCA proměnných. Pak lineární závislost velmi klesá.

Distribuční funkce PCA proměnných jsou odlišné od těch původních.

In [ ]:
pos = 1
fig = plt.figure(figsize=(12,16))
for i in data_std_pca.columns:
    ax = fig.add_subplot(7,2, pos)
    pos = pos + 1
    sns.histplot(data_std_pca[i],ax=ax, kde=True)

## Lineární model ze všech PCA proměnných 
Data si opět rozdělíme na na trénovací a testovací.

In [ ]:
X = np.array(data_std_pca.drop('MEDV',axis=1))
Y = np.array(data_std_pca['MEDV'])
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

Vytvoříme lineární model

In [ ]:
lr = LinearRegression()
lr.fit(X_train, y_train)

Validace modelu pro trénovací data

In [ ]:
y_pred = lr.predict(X_train)
r2 = r2_score(y_train, y_pred)
rmse = np.sqrt(mean_squared_error(y_train, y_pred))

print(f"R2 score: {r2}")
print(f"RMSE: {rmse}")

Validace modelu pro testovací data

In [ ]:
y_pred = lr.predict(X_test)
r2 = r2_score(y_test, y_pred)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))

print(f"R2 score: {r2}")
print(f"RMSE: {rmse}")

Výsledný model z PCA proměnných je o něco lepší než původní lineární model z původních proměnných.

## Lineární model ze 6 PCA proměnných
PCA lze také použít pro redukci dimenzionality. 

Vytvoříme tedy model, který bude mít místo 13 vstupních proměnných pouze 6 proměnných.

In [ ]:
lr = LinearRegression()
lr.fit(X_train[:,0:6], y_train)

Validace modelu na trénovacích datech

In [ ]:
y_pred = lr.predict(X_train[:,0:6])
r2 = r2_score(y_train, y_pred)
rmse = np.sqrt(mean_squared_error(y_train, y_pred))

print(f"R2 score: {r2}")
print(f"RMSE: {rmse}")

Validace modelu na testovacích datech

In [ ]:
y_pred = lr.predict(X_test[:,0:6])
r2 = r2_score(y_test, y_pred)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))

print(f"R2 score: {r2}")
print(f"RMSE: {rmse}")

Přesnost redukovaného modelu je o podle očekávání o něco nižší. Na druhou stranu model je menší.